In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir('../input/petfinder-pawpularity-score'))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Make sure you have keras trained weights downloaded in input directory using https://www.kaggle.com/antoreepjana/tf-keras-pretrained-model-weights .This will be useful as we are not supposed to use internet while running inference 

In [ ]:
rootDir = "../input/petfinder-pawpularity-score/"


In [ ]:
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import cv2
import matplotlib.pyplot as plt 
from os.path import basename
%matplotlib inline


In [ ]:
trainf = pd.read_csv(rootDir + "train.csv")
testf = pd.read_csv(rootDir + "test.csv")


In [ ]:
trainf['imagename'] = trainf['Id'].apply(lambda x : str(x) + ".jpg")
testf['imagename'] = testf['Id'].apply(lambda x : str(x) + ".jpg")

In [ ]:
i = 10
imgpath = rootDir + "train/" + os.listdir(rootDir + "train/")[i]
imgname = basename(imgpath)
print(f"image name is {imgname}")
pscore = trainf[trainf['imagename']== imgname]['Pawpularity'].unique()[0]
print(f"popularity score is {pscore}")
img = Image.open(imgpath)
img

In [ ]:
trainf[trainf['imagename']== imgname]

## Base CNN model

In [ ]:
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,BatchNormalization,GlobalAveragePooling2D,Flatten,Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16,ResNet152
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
from sklearn.model_selection import train_test_split

In [ ]:
train_df,valid_df = train_test_split(trainf,test_size = 0.2,random_state  = 1)
train_df = train_df.reset_index(drop = True)
valid_df = valid_df.reset_index(drop = True)
print(train_df.shape,valid_df.shape)


In [ ]:
train_df.head()

In [ ]:
train_df.columns

### Below written code can be used to utilize default keras Imagegenerator class but we can't add tabular features in it hence we would be writing a custome class. Also the following model is a vgg16 based transfer learning model

In [ ]:
# def transferlearn_model():
#     imgsize = 224 ## using vgg 
#     vgg_model_wtl = VGG16(weights = "imagenet",input_shape = (imgsize,imgsize,3), include_top = False ) 
#     layername = "block3_conv3"
#     mymodel = Model(inputs = vgg_model_wtl.input,outputs = vgg_model_wtl.get_layer(layername).output)
#     newmodel1 = Sequential()
#     newmodel1.add(mymodel)

#     newmodel1.add(Conv2D(128,(3,3),activation='relu',padding='same'))
#     newmodel1.add(MaxPooling2D((2,2),padding='same'))

#     newmodel1.add(Conv2D(128,(3,3),activation='relu',padding='same'))
#     newmodel1.add(MaxPooling2D((2,2),padding='same'))


#     newmodel1.add(Conv2D(256,(3,3),activation='relu',padding='same'))
#     newmodel1.add(MaxPooling2D((2,2),padding='same'))

#     newmodel1.add(GlobalAveragePooling2D())
#     newmodel1.add(Dense(64,activation='relu'))
#     newmodel1.add(BatchNormalization())

#     newmodel1.add(Dense(1,activation='linear'))

#     newmodel1.layers[0].trainable=False
    
#     newmodel1.compile(loss='mean_squared_error', optimizer='adam',
#                   metrics=['mse'])
#     return newmodel1
    
# model2train = transferlearn_model()

# ### train and valid
# train_datagen = ImageDataGenerator(
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True)

# train_generator = train_datagen.flow_from_dataframe(
#         dataframe=train_df,
#         directory=rootDir + "train/",
#         x_col="imagename",
#         y_col='Pawpularity', ## need not one hot encode 
#         target_size=(224, 224),
#         batch_size=32,
#         class_mode='raw')

# valid_datagen = ImageDataGenerator(rescale=1./255)


# validation_generator = valid_datagen.flow_from_dataframe(
#         dataframe=valid_df,
#         directory=rootDir + "train/",
#         x_col="imagename",
#         y_col='Pawpularity',
#         target_size=(224, 224),
#         batch_size=32,
#         class_mode='raw')


# ES_PATIENCE = 10
# RLROP_PATIENCE = 3
# DECAY_DROP = 0.4

# es = EarlyStopping(monitor='val_loss', mode='min', 
#                    patience=ES_PATIENCE, restore_best_weights=True, verbose=1)
# rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', 
#                           patience=RLROP_PATIENCE, factor=DECAY_DROP, min_lr=1e-6, verbose=1)

# callback_list = [es,rlrop]

# RunRes = model2train.fit_generator(generator=train_generator,
#                     validation_data=validation_generator,epochs = 5,callbacks=callback_list)

# RunRes.history.keys()
# fig,axes = plt.subplots(2,2,figsize = (8,8))
# axes[0,0].plot(RunRes.history['loss'])
# axes[0,1].plot(RunRes.history['val_loss'])
# axes[1,0].plot(RunRes.history['mse'])
# axes[1,1].plot(RunRes.history['val_mse'])
# plt.show()

## Custom class to add Image features

In [ ]:
class customImageGen(tf.keras.utils.Sequence):
    def __init__(self,imgdir,_dataF,X_col,otherfeats,target,batch_size = 32,dim = (256,256,3),shuffle = True,n_channels = 1,n_classes = 1,flag = "train"):
        self.imgdir  = imgdir 
        self._dataF = _dataF 
        self.target = target
        self.dim = dim 
        self.shuffle = shuffle 
        self.X_col = X_col
        self.otherfeats = otherfeats
        self.n_channels = n_channels
        self.batch_size = batch_size 
        self.indexes = self._dataF.index
        self.n_classes = n_classes
        self.flag = flag
        
        
    def augmen(self,samples):
        if self.flag == "train":
            datagentrain = ImageDataGenerator(rescale=1. / 255.0,
                                  horizontal_flip = True,
                                  vertical_flip = False, 
                                  height_shift_range= 0.1, 
                                  width_shift_range=0.1, 
                                  rotation_range=20, 
                                  shear_range = 0.1,
                                  zoom_range=0.1)

            it = datagentrain.flow(samples, batch_size=1)
        elif self.flag == "val":
            datagenval = ImageDataGenerator(rescale=1. / 255.0)

            it = datagenval.flow(samples, batch_size=1)
        return it.next() 
    
    def __len__(self):
        'Denotes number of batches per epoch'
        return int(np.floor(len(self.indexes)/ self.batch_size))
    def __getitem__(self,index):
        "Generate one batch of data"
        indexes = self.indexes[index*self.batch_size:(index + 1)*self.batch_size]
        
        ## find List of ids 
        list_IDs_temp = [self.indexes[k] for k in indexes]
        x,otherfeats,y = self.__data__generation(list_IDs_temp)
        
        return [x,otherfeats],y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.indexes))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data__generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size,self.n_classes), dtype=int)
        otherfeats = np.empty((self.batch_size,len(self.otherfeats)), dtype=int)
        
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            #img=cv2.imread(train_data_dir + ID)
            img_initial = image.load_img(self.imgdir + self._dataF.loc[ID][self.X_col])
            img = img_initial.resize((self.dim[:-1]))
            img_array = image.img_to_array(img)
            
            samples = np.expand_dims(img_array, 0)
            
            aug_samples = self.augmen(samples)

            
            X[i,] = aug_samples

            # Store class
            y[i] = self._dataF.loc[ID][self.target] #self.labels[ID]  #y[i]
            
            otherfeats[i] = self._dataF[self.otherfeats].iloc[ID].values
            
            
            
#         print(f"X shape is {X.shape} and otherf shape is {otherfeats.shape}")

        return X,otherfeats,y
    

In [ ]:
### Additional features
otherfeats = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']

In [ ]:
imgg = customImageGen(rootDir + "train/",train_df,'imagename',otherfeats,"Pawpularity",batch_size = 1,dim = (256,256,3),shuffle = True,n_channels = 3,flag = 'train')

In [ ]:
imgg.__getitem__(0)[0][0].shape

Let's check if it is working fine

In [ ]:
plt.imshow(imgg.__getitem__(0)[0][0][0])

print(imgg.__getitem__(0)[0][1],imgg.__getitem__(0)[1])

In [ ]:
batch_size = 32

Train and validation generator

In [ ]:
trainGen = customImageGen(rootDir + "train/",train_df,'imagename',otherfeats,"Pawpularity",batch_size = batch_size,dim = \
                          (256,256,3),shuffle = True,n_channels = 3,flag = 'train')
valGen = customImageGen(rootDir + "train/",valid_df,'imagename',otherfeats,"Pawpularity",batch_size = batch_size,dim = \
                        (256,256,3),shuffle = True,n_channels = 3,flag = 'val')

In [ ]:
valGen.indexes.shape +trainGen.indexes.shape


In [ ]:
# valGen.__getitem__(1)[0][0].shape,valGen.__getitem__(1)[0][1].shape

from numpy.random import seed
seed(1)

### custom model to utlize both CNN and ANN

In [ ]:
input_feat_tensor = tf.keras.Input(shape=len(otherfeats),name='feat_input')
# imgsize = 224
import tensorflow.keras.backend as K

def root_mean_squared_error(y_true, y_pred):
    y_true = float(y_true)
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 
### CNN model for images
imgsize = 128
def cnnmod():
    # Create base model
    #base_model =VGG16(weights = "imagenet",input_shape = (imgsize,imgsize,3), include_top = False ) 
    base_model = ResNet152(weights='../input/tf-keras-pretrained-model-weights/No Top/resnet152_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False,input_tensor=keras.Input(shape=(imgsize,imgsize,3)))
    # Freeze base model
    base_model.trainable = False
    # Create new model on top.
    inputs = keras.Input(shape=(imgsize, imgsize, 3))
    x = base_model(inputs, training=False)
#     x = Conv2D(128,(3,3),activation='relu',padding='same')(x)
#     x = MaxPooling2D((2,2),padding='same')(x)
#     x = Conv2D(128,(3,3),activation='relu',padding='same')(x)
#     x = MaxPooling2D((2,2),padding='same')(x)
#     x = Conv2D(256,(3,3),activation='relu',padding='same')(x)
#     x = MaxPooling2D((2,2),padding='same')(x)
#     x = GlobalAveragePooling2D()(x)
#     x = Dense(64,activation='relu')(x)
#     x = BatchNormalization()(x)
#     x = keras.layers.GlobalAveragePooling2D()(x)
    x = Dense(2048,activation = 'relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1000, activation='relu')(x)
    x = Dropout(0.5)(x)
    #x = Dense(600, activation='relu')(x)
    #x = Dropout(0.5)(x)
    x = Dense(300, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    x = GlobalAveragePooling2D()(x)
#     x = MaxPooling2D()(x)
#     x = Flatten()(x)
    x = Dense(500,activation='relu', name='final_outputcnn')(x)
#     final_output = Dense(50, activation='relu', name='final_output')(x)
#     outputs = Dense(8,activation='relu',name= "lastcnnlayer")(x)
    model = tf.keras.Model(inputs, x)
    return model
createcnn = cnnmod()

######## MLP model for tabular featues
def annmodel():
    input_feat_tensor = tf.keras.Input(shape=len(otherfeats),name='feat_input')
    x = Dense(8,activation = 'relu',kernel_initializer=tf.keras.initializers.HeNormal())(input_feat_tensor)
    x = Dropout(0.5)(x)
    #x = Dense(6,activation = 'relu',kernel_initializer=tf.keras.initializers.HeNormal())(input_feat_tensor)
    #x = Dropout(0.5)(x)
    model = tf.keras.Model(input_feat_tensor,x)
    return model
createann = annmodel()
# conlayer = tf.keras.layers.Concatenate(axis=1)([createcnn.output,otherfeatsar ])
# conlayer = tf.keras.layers.concatenate([createcnn.output, input_feat_tensor])
conlayer = tf.keras.layers.concatenate([createcnn.output, createann.output])
conlayer = BatchNormalization(name = "conclayernorm")(conlayer)
conlayer = Dense(200,name = "nonlinint",activation = 'relu')(conlayer)
conlayer = Dropout(0.5)(conlayer)
conlayer = Dense(100,name = "nonlinint2",activation = 'relu')(conlayer)
conlayer = Dropout(0.5)(conlayer)
z  = Dense(1,activation='linear', name='final_output')(conlayer)
# finalmod = Model(inputs = [createcnn.input,keras.layers.Input(shape=(12,))],outputs = z )
# finalmod = Model(inputs = [createcnn.input,input_feat_tensor],outputs = z )
finalmod = Model(inputs = [createcnn.input,createann.input],outputs = z )
finalmod.compile(loss=[root_mean_squared_error], optimizer='adam',metrics=['mse']) ## custom rmse loss
    
    
    


In [ ]:
finalmod.summary()

In [ ]:
createcnn.output,createann.output,createcnn.input,createann.input

### training model and saving weights. I am using few callbacks 
* Early stopping : To stop training if there is no decrease in validation loss after 8 epochs 
* Reduce learning rate on plateau : To decrease LR as training curve hits a plateau
* Model checkpoint : To save model weights

In [ ]:
import datetime ,re
def cleaner(x):
    x = re.sub("[^0-9]","_",x)
    return x


In [ ]:
dt2attch = cleaner(str(datetime.datetime.now()))
filepath="./pawweights.best_{epoch:02d}-{val_loss:.2f}" + "_" + dt2attch +".hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', 
                             verbose=1, save_best_only=True, mode='auto')

ES_PATIENCE = 10
RLROP_PATIENCE = 5
DECAY_DROP = 0.3
es = EarlyStopping(monitor='val_loss', mode='min', 
                   patience=ES_PATIENCE, restore_best_weights=True, verbose=1)
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', 
                          patience=RLROP_PATIENCE, factor=DECAY_DROP, min_lr=1e-6, verbose=1)

callback_list = [checkpoint,es,rlrop]

In [ ]:
valGen.indexes.shape[0],trainGen.indexes.shape[0],valGen.indexes.shape[0]//batch_size,trainGen.indexes.shape[0]//batch_size


In [ ]:
RunRes = finalmod.fit_generator(trainGen,steps_per_epoch=trainGen.indexes.shape[0]//batch_size,validation_data=valGen,
                                validation_steps=valGen.indexes.shape[0]//batch_size,
                                epochs = 50,callbacks=callback_list,verbose=1)
print(RunRes.history.keys())
fig,axes = plt.subplots(2,2,figsize = (8,8))
axes[0,0].plot(RunRes.history['loss'])
axes[0,1].plot(RunRes.history['val_loss'])
axes[1,0].plot(RunRes.history['mse'])
axes[1,1].plot(RunRes.history['val_mse'])
plt.show()
